# COBOL to Docs v1.0 - Notebook de Exemplo

Este notebook demonstra como usar o sistema COBOL to Docs v1.0 com todas as suas funcionalidades avançadas:

- **RAG (Retrieval-Augmented Generation)**: Sistema de recuperação e geração aumentada
- **Aprendizado Automático**: Base de conhecimento que evolui
- **Múltiplos Providers**: Enhanced Mock, LuzIA, GitHub Copilot
- **Análise Profissional Sênior**: Baseada em feedback de especialistas COBOL

## Pré-requisitos

```bash
pip install jupyter pandas matplotlib seaborn
```

## 1. Configuração Inicial

Primeiro, vamos configurar o ambiente e importar as bibliotecas necessárias.

In [ ]:
import sys
import os
import json
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Adicionar src ao path
sys.path.insert(0, 'src')

# Importar biblioteca principal
from src.library.cobol_analysis_lib import COBOLAnalysisLibrary
from src.parsers.cobol_parser_original import COBOLParser, CobolProgram

# Configurar visualizações
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("✅ Ambiente configurado com sucesso!")

## 2. Inicialização da Biblioteca

Vamos inicializar a biblioteca COBOL to Docs com todas as funcionalidades.

In [ ]:
# Inicializar biblioteca
library = COBOLAnalysisLibrary(
    config_path="config/config.yaml",
    output_dir="output_notebook"
)

print("📊 Status dos Providers:")
provider_status = library.get_provider_status()
for provider, status in provider_status.items():
    status_icon = "✅" if status == "disponível" else "❌"
    print(f"  {status_icon} {provider}: {status}")

print(f"\n🎯 Modelos Disponíveis: {library.get_available_models()}")
print(f"📋 Estratégias: {library.get_available_strategies()}")

## 3. Exemplo de Análise Individual

Vamos analisar um programa COBOL individual usando diferentes estratégias.

In [ ]:
# Exemplo de programa COBOL
sample_cobol = """
       IDENTIFICATION DIVISION.
       PROGRAM-ID. LHAN0542.
       
       ENVIRONMENT DIVISION.
       INPUT-OUTPUT SECTION.
       FILE-CONTROL.
           SELECT CLIENTE-FILE ASSIGN TO 'CLIENTE.DAT'
           ORGANIZATION IS SEQUENTIAL
           ACCESS MODE IS SEQUENTIAL.
       
       DATA DIVISION.
       FILE SECTION.
       FD  CLIENTE-FILE.
       01  CLIENTE-RECORD.
           05 CLIENTE-ID       PIC 9(10).
           05 CLIENTE-NOME     PIC X(50).
           05 CLIENTE-CPF      PIC 9(11).
           05 CLIENTE-SALDO    PIC 9(15)V99.
           05 CLIENTE-STATUS   PIC X(01).
       
       WORKING-STORAGE SECTION.
       01  WS-CONTADOR         PIC 9(05) VALUE ZERO.
       01  WS-TOTAL-SALDO      PIC 9(15)V99 VALUE ZERO.
       01  WS-FIM              PIC X(01) VALUE 'N'.
       
       PROCEDURE DIVISION.
       MAIN-PROCESS.
           OPEN INPUT CLIENTE-FILE
           PERFORM PROCESSAR-CLIENTES UNTIL WS-FIM = 'S'
           CLOSE CLIENTE-FILE
           DISPLAY 'TOTAL PROCESSADO: ' WS-CONTADOR
           DISPLAY 'SALDO TOTAL: ' WS-TOTAL-SALDO
           STOP RUN.
       
       PROCESSAR-CLIENTES.
           READ CLIENTE-FILE
               AT END MOVE 'S' TO WS-FIM
               NOT AT END
                   IF CLIENTE-STATUS = 'A'
                       ADD 1 TO WS-CONTADOR
                       ADD CLIENTE-SALDO TO WS-TOTAL-SALDO
                       DISPLAY 'CLIENTE: ' CLIENTE-NOME
                   END-IF
           END-READ.
"""

# Analisar com estratégia expert_analysis
print("🔍 Analisando programa com estratégia expert_analysis...")
result = library.analyze_content(
    content=sample_cobol,
    program_name="LHAN0542_EXEMPLO",
    model="enhanced_mock",
    analysis_strategy="expert_analysis"
)

if result['success']:
    print(f"✅ Análise bem-sucedida!")
    print(f"📄 Documentação: {result['documentation_path']}")
    print(f"🎯 Tokens: {result['tokens_used']}")
    print(f"⏱️ Tempo: {result['processing_time']:.2f}s")
    print(f"🤖 Provider: {result['provider']}")
else:
    print(f"❌ Falha na análise: {result.get('error', 'Erro desconhecido')}")

## 4. Análise em Lote

Demonstração de análise em lote de múltiplos programas.

In [ ]:
# Verificar se existem arquivos de exemplo
examples_dir = Path("examples")
if examples_dir.exists():
    cobol_files = list(examples_dir.glob("*.cbl")) + list(examples_dir.glob("*.cob"))
    
    if cobol_files:
        print(f"📁 Encontrados {len(cobol_files)} arquivos COBOL em examples/")
        
        # Analisar em lote
        batch_result = library.batch_analyze(
            file_paths=[str(f) for f in cobol_files[:3]],  # Máximo 3 para demo
            model="enhanced_mock",
            analysis_strategy="expert_analysis"
        )
        
        if batch_result['success']:
            print(f"✅ Análise em lote concluída!")
            print(f"📊 Total: {batch_result['total_files']} arquivos")
            print(f"✅ Sucessos: {batch_result['successful_files']}")
            print(f"❌ Falhas: {batch_result['failed_files']}")
            
            # Salvar resultados para análise
            batch_data = batch_result
        else:
            print(f"❌ Falha na análise em lote")
    else:
        print("📁 Nenhum arquivo COBOL encontrado em examples/")
else:
    print("📁 Diretório examples/ não encontrado")
    # Criar dados simulados para demonstração
    batch_data = {
        'success': True,
        'total_files': 5,
        'successful_files': 4,
        'failed_files': 1,
        'results': [
            {'success': True, 'tokens_used': 1250, 'processing_time': 2.3},
            {'success': True, 'tokens_used': 980, 'processing_time': 1.8},
            {'success': True, 'tokens_used': 1450, 'processing_time': 2.7},
            {'success': True, 'tokens_used': 1100, 'processing_time': 2.1},
            {'success': False, 'tokens_used': 0, 'processing_time': 0.5}
        ]
    }
    print("📊 Usando dados simulados para demonstração")

## 5. Visualização de Estatísticas

Vamos criar visualizações das estatísticas de análise.

In [ ]:
# Obter estatísticas do sistema
stats = library.get_statistics()

# Criar DataFrame com resultados
if 'batch_data' in locals() and batch_data['success']:
    results_df = pd.DataFrame([
        {
            'programa': f'PROG_{i+1}',
            'sucesso': r['success'],
            'tokens': r['tokens_used'],
            'tempo': r['processing_time']
        }
        for i, r in enumerate(batch_data['results'])
    ])
    
    # Visualizações
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    fig.suptitle('COBOL to Docs v1.0 - Estatísticas de Análise', fontsize=16, fontweight='bold')
    
    # 1. Taxa de Sucesso
    success_data = results_df['sucesso'].value_counts()
    axes[0,0].pie(success_data.values, labels=['Sucesso', 'Falha'], autopct='%1.1f%%', 
                  colors=['#2ecc71', '#e74c3c'])
    axes[0,0].set_title('Taxa de Sucesso das Análises')
    
    # 2. Distribuição de Tokens
    successful_results = results_df[results_df['sucesso'] == True]
    if not successful_results.empty:
        axes[0,1].bar(successful_results['programa'], successful_results['tokens'], 
                      color='#3498db', alpha=0.7)
        axes[0,1].set_title('Tokens Utilizados por Programa')
        axes[0,1].set_ylabel('Tokens')
        axes[0,1].tick_params(axis='x', rotation=45)
    
    # 3. Tempo de Processamento
    if not successful_results.empty:
        axes[1,0].bar(successful_results['programa'], successful_results['tempo'], 
                      color='#9b59b6', alpha=0.7)
        axes[1,0].set_title('Tempo de Processamento por Programa')
        axes[1,0].set_ylabel('Tempo (segundos)')
        axes[1,0].tick_params(axis='x', rotation=45)
    
    # 4. Estatísticas Gerais
    stats_text = f"""
    Programas Analisados: {stats['programs_analyzed']}
    Análises Bem-sucedidas: {stats['successful_analyses']}
    Análises Falharam: {stats['failed_analyses']}
    Total de Tokens: {stats['total_tokens_used']}
    Tempo Total: {stats['total_processing_time']:.2f}s
    """
    axes[1,1].text(0.1, 0.5, stats_text, fontsize=12, verticalalignment='center')
    axes[1,1].set_title('Estatísticas Gerais')
    axes[1,1].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    # Mostrar DataFrame
    print("\n📊 Resultados Detalhados:")
    display(results_df)
else:
    print("📊 Dados insuficientes para visualização")

## 6. Exploração da Base de Conhecimento RAG

Vamos explorar as funcionalidades do sistema RAG.

In [ ]:
# Obter estatísticas RAG
rag_stats = library.get_rag_statistics()

print("🧠 Estatísticas do Sistema RAG:")
if rag_stats.get('rag_enabled', False):
    print(f"  📚 Base de Conhecimento: {rag_stats.get('knowledge_base_size', 'N/A')}")
    print(f"  🔄 Total de Interações: {rag_stats.get('total_interactions', 'N/A')}")
    print(f"  🎯 Padrões Descobertos: {rag_stats.get('pattern_discoveries', 'N/A')}")
    
    # Mostrar progresso do aprendizado
    learning_progress = library.get_learning_progress()
    if learning_progress.get('learning_enabled', False):
        print(f"\n🎓 Progresso do Aprendizado:")
        print(f"  🔍 Padrões Aprendidos: {learning_progress.get('patterns_learned', 'N/A')}")
        print(f"  📈 Precisão: {learning_progress.get('accuracy', 'N/A')}")
        print(f"  📊 Feedback Incorporado: {learning_progress.get('feedback_incorporated', 'N/A')}")
else:
    print("  ❌ RAG não está habilitado")

# Demonstrar como o sistema aprende
print("\n🔬 Demonstração de Aprendizado:")
print("O sistema RAG aprende automaticamente com cada análise realizada:")
print("  1. Identifica padrões no código COBOL")
print("  2. Correlaciona com análises bem-sucedidas")
print("  3. Atualiza a base de conhecimento")
print("  4. Melhora análises futuras")

## 7. Comparação de Estratégias de Análise

Vamos comparar diferentes estratégias de análise no mesmo programa.

In [ ]:
# Programa de teste mais complexo
complex_cobol = """
       IDENTIFICATION DIVISION.
       PROGRAM-ID. LHAN0544.
       
       ENVIRONMENT DIVISION.
       INPUT-OUTPUT SECTION.
       FILE-CONTROL.
           SELECT TRANSACAO-FILE ASSIGN TO 'TRANSACAO.DAT'.
           SELECT CADOC-FILE ASSIGN TO 'CADOC.DAT'.
       
       DATA DIVISION.
       FILE SECTION.
       FD  TRANSACAO-FILE.
       01  TRANSACAO-RECORD.
           05 TRANS-ID         PIC 9(15).
           05 TRANS-TIPO       PIC X(03).
           05 TRANS-VALOR      PIC 9(13)V99.
           05 TRANS-CONTA      PIC 9(10).
           05 TRANS-INSTITUICAO PIC 9(04).
       
       WORKING-STORAGE SECTION.
       01  WS-CADOC-VARS.
           05 WS-INSTITUICAO   PIC 9(04).
           05 WS-DOCUMENTO     PIC X(20).
           05 WS-STATUS        PIC X(01).
       
       PROCEDURE DIVISION.
       MAIN-PROCESS.
           PERFORM VALIDAR-CADOC
           IF WS-STATUS = 'V'
               PERFORM PROCESSAR-TRANSACOES
           ELSE
               DISPLAY 'ERRO: VALIDACAO CADOC FALHOU'
           END-IF
           STOP RUN.
       
       VALIDAR-CADOC.
           MOVE 0033 TO WS-INSTITUICAO
           IF WS-INSTITUICAO = 0033 OR 1508
               MOVE 'V' TO WS-STATUS
           ELSE
               MOVE 'I' TO WS-STATUS
           END-IF.
"""

# Testar diferentes estratégias
strategies = ['expert_analysis', 'comprehensive', 'rag_enhanced']
strategy_results = {}

for strategy in strategies:
    print(f"\n🔍 Testando estratégia: {strategy}")
    
    result = library.analyze_content(
        content=complex_cobol,
        program_name=f"LHAN0544_{strategy.upper()}",
        model="enhanced_mock",
        analysis_strategy=strategy
    )
    
    if result['success']:
        strategy_results[strategy] = {
            'tokens': result['tokens_used'],
            'time': result['processing_time'],
            'success': True
        }
        print(f"  ✅ Sucesso - Tokens: {result['tokens_used']}, Tempo: {result['processing_time']:.2f}s")
    else:
        strategy_results[strategy] = {
            'tokens': 0,
            'time': 0,
            'success': False
        }
        print(f"  ❌ Falha")

# Visualizar comparação
if strategy_results:
    comparison_df = pd.DataFrame(strategy_results).T
    
    print("\n📊 Comparação de Estratégias:")
    display(comparison_df)
    
    # Gráfico de comparação
    successful_strategies = comparison_df[comparison_df['success'] == True]
    
    if not successful_strategies.empty:
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
        
        # Tokens por estratégia
        ax1.bar(successful_strategies.index, successful_strategies['tokens'], 
                color=['#e74c3c', '#3498db', '#2ecc71'])
        ax1.set_title('Tokens Utilizados por Estratégia')
        ax1.set_ylabel('Tokens')
        ax1.tick_params(axis='x', rotation=45)
        
        # Tempo por estratégia
        ax2.bar(successful_strategies.index, successful_strategies['time'], 
                color=['#e74c3c', '#3498db', '#2ecc71'])
        ax2.set_title('Tempo de Processamento por Estratégia')
        ax2.set_ylabel('Tempo (segundos)')
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        plt.show()

## 8. Análise de Arquivos de Saída

Vamos examinar os arquivos gerados pelo sistema.

In [ ]:
# Listar arquivos gerados
output_dir = Path("output_notebook")

if output_dir.exists():
    print("📁 Arquivos gerados:")
    
    # Relatórios markdown
    md_files = list(output_dir.glob("*.md"))
    print(f"\n📄 Relatórios Markdown ({len(md_files)}):")
    for md_file in md_files:
        size_kb = md_file.stat().st_size / 1024
        print(f"  - {md_file.name} ({size_kb:.1f} KB)")
    
    # JSONs de auditoria
    json_dir = output_dir / "ai_responses"
    if json_dir.exists():
        json_files = list(json_dir.glob("*.json"))
        print(f"\n📊 JSONs de Auditoria ({len(json_files)}):")
        for json_file in json_files:
            size_kb = json_file.stat().st_size / 1024
            print(f"  - {json_file.name} ({size_kb:.1f} KB)")
    
    # Estatísticas
    stats_dir = output_dir / "statistics"
    if stats_dir.exists():
        stats_files = list(stats_dir.glob("*.json"))
        print(f"\n📈 Arquivos de Estatísticas ({len(stats_files)}):")
        for stats_file in stats_files:
            size_kb = stats_file.stat().st_size / 1024
            print(f"  - {stats_file.name} ({size_kb:.1f} KB)")
    
    # Mostrar exemplo de conteúdo
    if md_files:
        print(f"\n📖 Exemplo de conteúdo ({md_files[0].name}):")
        with open(md_files[0], 'r', encoding='utf-8') as f:
            content = f.read()
            # Mostrar primeiras 500 caracteres
            print(content[:500] + "..." if len(content) > 500 else content)
else:
    print("📁 Diretório de saída não encontrado")

## 9. Conclusões e Próximos Passos

Este notebook demonstrou as principais funcionalidades do COBOL to Docs v1.0:

### Funcionalidades Demonstradas

1. **Análise Individual**: Análise detalhada de programas COBOL únicos
2. **Análise em Lote**: Processamento eficiente de múltiplos arquivos
3. **Múltiplas Estratégias**: Diferentes abordagens de análise
4. **Sistema RAG**: Base de conhecimento que evolui automaticamente
5. **Visualizações**: Gráficos e estatísticas detalhadas
6. **Auditoria Completa**: JSONs de rastreabilidade

### Vantagens do Sistema

- **Análise Profissional Sênior**: Baseada em feedback de especialistas
- **Aprendizado Contínuo**: Melhora com cada análise
- **Flexibilidade**: Múltiplos providers e estratégias
- **Transparência**: Auditoria completa do processo
- **Escalabilidade**: Análise em lote eficiente

### Próximos Passos

1. Integrar com providers reais (LuzIA, GitHub Copilot)
2. Expandir base de conhecimento CADOC
3. Implementar feedback de usuários
4. Adicionar métricas de qualidade
5. Desenvolver API REST

### Como Usar em Produção

```bash
# Análise individual
python main.py programa.cbl

# Análise em lote
python main.py *.cbl --output ./relatorios

# Via CLI
python -m src.client.cli_client analyze programa.cbl --model luzia
```